In [2]:
!pip install sounddevice

In [5]:
!sudo apt-get install libportaudio2


[sudo] password for gautam: 


In [6]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import sounddevice as sd
import numpy as np
import queue
import sys
import os
import threading

In [7]:
# Function to transcribe audio
def transcribe_audio(audio_input):
    # Load pre-trained model and processor
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

    # Tokenize audio waveform
    input_values = processor(audio_input, return_tensors="pt").input_values

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode output
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription


In [8]:
# Function to record audio from the microphone
def record_audio(q):
    def callback(indata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        q.put(indata.copy())

    q.put(None)  # Sentinel to indicate start of recording
    with sd.InputStream(callback=callback):
        while True:
            data = q.get()
            if data is None:
                break
            yield data

In [9]:
# Function to continuously transcribe audio input
def listen_and_transcribe(q):
    for audio_chunk in record_audio(q):
        if audio_chunk is not None:
            transcription = transcribe_audio(audio_chunk)
            if transcription.strip():
                print("Transcription:", transcription)


In [10]:

# Main function
def main():
    q = queue.Queue()
    listening_thread = threading.Thread(target=listen_and_transcribe, args=(q,))
    listening_thread.start()
    print("Listening... Press Ctrl+C to stop.")
    try:
        listening_thread.join()
    except KeyboardInterrupt:
        print("\nStopping...")
        q.put(None)

if __name__ == "__main__":
    main()


Listening... Press Ctrl+C to stop.
